---
# Exercício Titanic - kNN, Regressão Logística
---
**Machine Learning em Projetos (Erick Muzart e Fernando melo)**   
Tópicos:
- One-hot Encoder get-dummies
- Knn
- Regressão logística
- Matriz de confusão

<br>
------------------------------------------------
   
Usando dados do Titanic, descritos em: https://www.openml.org/d/40945

## Carregar bibliotecas

In [2]:
# Importar bibliotecas pandas, plot_confusion_matrix, train_test_split, matplotlib, metrics
import pandas as pd

## Análise exploratória dos dados

In [3]:
# Ler dados do Titanic a partir do site Openml.org
df = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl', na_values='?')
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [4]:
nan_series = df.isna().sum()/len(df)
nan_series

pclass       0.000000
survived     0.000000
name         0.000000
sex          0.000000
age          0.200917
sibsp        0.000000
parch        0.000000
ticket       0.000000
fare         0.000764
cabin        0.774637
embarked     0.001528
boat         0.628724
body         0.907563
home.dest    0.430863
dtype: float64

In [5]:
highest_nans = nan_series[nan_series > 0.5].index
highest_nans

Index(['cabin', 'boat', 'body'], dtype='object')

In [6]:
#cabin feature
#apply: people with the same ticket stay in the same cabin
df_cabin_nan = df.loc[df.cabin.isna()]
df_cabin_nan

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0,0,0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay"
13,1,1,"Barber, Miss. Ellen 'Nellie'",female,26.0,0,0,19877,78.8500,NaN,S,6,NaN,NaN
15,1,0,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.9250,NaN,S,NaN,NaN,"New York, NY"
23,1,1,"Bidois, Miss. Rosalie",female,42.0,0,0,PC 17757,227.5250,NaN,C,4,NaN,NaN
25,1,0,"Birnbaum, Mr. Jakob",male,25.0,0,0,13905,26.0000,NaN,C,NaN,148.0,"San Francisco, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [7]:
df.cabin.describe()

count             295
unique            186
top       C23 C25 C27
freq                6
Name: cabin, dtype: object

In [8]:
df_cabin_notnan = df.loc[df.cabin.isna()==False]
df_cabin_notnan

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,3,1,"Sandstrom, Miss. Marguerite Rut",female,4.0000,1,1,PP 9549,16.7000,G6,S,13,NaN,NaN
1217,3,0,"Soholt, Mr. Peter Andreas Lauritz Andersen",male,19.0000,0,0,348124,7.6500,F G73,S,NaN,NaN,NaN
1230,3,0,"Strom, Miss. Telma Matilda",female,2.0000,0,1,347054,10.4625,G6,S,NaN,NaN,NaN
1231,3,0,"Strom, Mrs. Wilhelm (Elna Matilda Persson)",female,29.0000,1,1,347054,10.4625,G6,S,NaN,NaN,NaN


In [9]:
df.sort_values('ticket').head(20)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
67,1,1,"Cherry, Miss. Gladys",female,30.0,0,0,110152,86.5000,B77,S,8,NaN,"London, England"
245,1,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,110152,86.5000,B77,S,8,NaN,"London Vancouver, BC"
195,1,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.5000,B79,S,8,NaN,NaN
289,1,1,"Taussig, Miss. Ruth",female,18.0,0,2,110413,79.6500,E68,S,8,NaN,"New York, NY"
291,1,1,"Taussig, Mrs. Emil (Tillie Mandelbaum)",female,39.0,1,1,110413,79.6500,E67,S,8,NaN,"New York, NY"
290,1,0,"Taussig, Mr. Emil",male,52.0,1,1,110413,79.6500,E67,S,NaN,NaN,"New York, NY"
232,1,0,"Porter, Mr. Walter Chamberlain",male,47.0,0,0,110465,52.0000,C110,S,NaN,207.0,"Worcester, MA"
74,1,0,"Clifford, Mr. George Quincy",male,NaN,0,0,110465,52.0000,A14,S,NaN,NaN,"Stoughton, MA"
194,1,0,"Maguire, Mr. John Edward",male,30.0,0,0,110469,26.0000,C106,S,NaN,NaN,"Brockton, MA"
34,1,0,"Borebank, Mr. John James",male,42.0,0,0,110489,26.5500,D22,S,NaN,NaN,"London / Winnipeg, MB"


In [ ]:
from statistics import mode

for i, row in df_cabin_nan.iterrows():
    cabin = df[(df['cabin'].isna() == False) 
                & (df['ticket']==row.ticket) 
                & (df['ticket'].isna() == False)].cabin
    if len(cabin) > 0:
        cabin_list = list(cabin)
        print(df.iloc[i].cabin, ' - ', mode(cabin_list), ' - ', len(cabin))


In [ ]:
df = df.drop(highest_nans, axis=1)
df

In [ ]:
nan_series = df.isna().sum()/len(df)
nan_series[nan_series > 0]

In [ ]:
df['home.dest'].unique()

In [ ]:
df = df.drop('home.dest', axis=1)
df

In [ ]:
nan_series = df.isna().sum()/len(df)
nan_series = nan_series[nan_series > 0]
nan_series

In [ ]:
df.embarked.unique()

In [ ]:
df.embarked.fillna(df.embarked.mode()[0], inplace=True)
df.embarked.value_counts()

In [ ]:
nan_series = df.isna().sum()/len(df)
nan_series = nan_series[nan_series > 0]
nan_series

In [ ]:
df[nan_series.index].describe()

In [ ]:
df.age.fillna(df.age.mean(), inplace=True)
df.age.isna().sum()

In [ ]:
df.fare.fillna(df.fare.mean(), inplace=True)
df.fare.isna().sum()

In [ ]:
df.isna().sum()

In [ ]:
df

In [ ]:
df = df.drop(['name', 'ticket'], axis=1)
df

In [ ]:
df.to_csv('titanic.csv')

In [ ]:
from autoML import AutoML
automl = AutoML(df, y_colname='survived')
automl.getResults()

In [ ]:
automl.getBestResult().features

In [ ]:
# Mostrar as dimensões e as primerias linhas do dataset


In [ ]:
# Mostra total de valores nulos por variável independente (features)


In [ ]:
# Fazer um corte no dataset com as colunas:'age', 'pclass', 'sex', 'embarked', 'survived' e eliminar dados nulos.


In [ ]:
# Mostrar novas dimensões do dataset após o corte


In [ ]:
# Verifica balanceamento das classses (value_counts())


In [ ]:
#mostra gráfico de barras do total de amostras por classe


## Pré-processamento

In [ ]:
# Atribuir valores para X e y

# Mostrar as dimensões de X e y


### Encoding variáveis (features) categoricas :
Se as variáveis não possuem uma ordem específica, geralmente a melhor opção é one hot-encoding (dummy encoding)

In [ ]:
# Cria X com as features codificadas com ".get_dummies" do Pandas.

# Mostra o resultado da codificação one-hot encoding


In [ ]:
# Dividir dados em treino e teste (70/30)


## Treinamento dos modelos

### Knn - K vizinhos mais próximos (K Nearest neighbors)

In [ ]:
# Importar o modelo knn Classifier


In [ ]:
# Instanciar modelo de regressão logística


In [ ]:
# Treinar o modelo com os dados de treinamento


In [ ]:
# Verificar a acurácia do modelo (.score)


In [ ]:
# Verificar o f1_score do modelo, quando as classes estão desbalenceadas


### Regressão Logística

In [ ]:
# Instanciar modelo de regressão logística


In [ ]:
# Treinar o modelo com os dados de treinamento


In [ ]:
# Verificar a acurácia do modelo (.score)


In [ ]:
# Verificar o f1_score do modelo, quando as classes estão desbalenceadas


## Avaliação da qualidade do melhor modelo

In [ ]:
# Importar a biblioteca (plot_confusion_matrix)


In [ ]:
# Mostrar a Matriz de Confusão do melhor modelo treinado


## Próximos passos
- Fazer normalização dos dados (Standard Scalar)   
- Treinar outros modelos:   
  - Árvore de decisão (Decision Tree Classifier)   
  - Floresta aleatória (Random Forest Classifier)

In [ ]:
# Árvore de decisão
clf = DecisionTreeClassifier(max_depth=4).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_class = RandomForestClassifier(n_estimators=120, max_depth=6, random_state=100).fit(X_train, y_train)
rf_class.score(X_test, y_test)